# NEST Tutorial | Part 2: Populations of neurons

Plot configuration

In [59]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import scienceplots

plt.style.use(["science"])
# mpl.rcParams["font.serif"] = ["Times New Roman"]


In [60]:
import nest
nest.ResetKernel()

### 1. Creating parameterised populations of nodes

We can specify the number of neurons to create by using the kwarg:

In [61]:
ndict = {"I_e": 200.0, "tau_m": 20.0}
neuronpop = nest.Create("iaf_psc_alpha", n=100, params=ndict)

print(neuronpop)

NodeCollection(metadata=None, model=iaf_psc_alpha, size=100, first=1, last=100)


There are three ways to conigure batches of neurons:
1. Parameterising the neurons at creation.
2. Using SetStatus() after creation
3. Setting the neuron model _before_ creation.

Always try to configure 3 > 2 > 1, since that order is most effective.

In [62]:
# Method 3

ndict = {"I_e": 200.0, "tau_m": 20.0}
nest.SetDefaults("iaf_psc_alpha", ndict)    # Setting parameters before init.

neuronpop1 = nest.Create("iaf_psc_alpha", n=100)
neuronpop2 = nest.Create("iaf_psc_alpha", n=100)
neuronpop3 = nest.Create("iaf_psc_alpha", n=100)

If batches of neurons should be of the same model but different parameters, the 
* CopyModel(existing, new, params) 

can be used:

In [63]:
edict = {"I_e": 200.0, "tau_m": 20.0}
nest.CopyModel("iaf_psc_alpha", "exc_iaf_psc_alpha")
nest.SetDefaults("exc_iaf_psc_alpha", edict)

In [64]:
idict = {"I_e": 300.0}
nest.CopyModel("iaf_psc_alpha", "inh_iaf_psc_alpha", params=idict)

The new models can be used to create neuron populations

In [65]:
epop1 = nest.Create("exc_iaf_psc_alpha", 100)
epop2 = nest.Create("exc_iaf_psc_alpha", 100)
ipop1 = nest.Create("inh_iaf_psc_alpha", 30)
ipop2 = nest.Create("inh_iaf_psc_alpha", 30)

Inhomogenous neuron parameters can be created by specifying a dict[str, List]

In [66]:
parameter_dict = {"I_e": [200.0, 150.0], "tau_m": 20.0, "V_m": [-77.0, -66.0]}
pop3 = nest.Create("iaf_psc_alpha", 2, params=parameter_dict)

print(pop3.get(["I_e", "tau_m", "V_m"]))

{'I_e': (200.0, 150.0), 'tau_m': (20.0, 20.0), 'V_m': (-77.0, -66.0)}


### 2. Setting parameters for populations of neurons

If some neurons in a population should have other values, such as some randomization, then list comprehension is the way to go

In [67]:
import numpy as np

def randomized_voltage(Vth, Vrest):
    return Vrest+(Vth-Vrest) * np.random.rand()

Vth=-55.
Vrest=-70.
dVms =  {"V_m": [randomized_voltage(Vth, Vrest) for i, x in enumerate(epop1)]}

epop1.set(dVms)

NEST also has in-built features which can be used for both node parameters and connection parameters (such as probability, weights and delays).

**Parameter complexity affects performance**


In [68]:
epop1.set({"V_m": Vrest + nest.random.uniform(0.0, Vth-Vrest)})

### 3. Generating populations of neurons with deterministic connections

In [69]:
import nest

pop1 = nest.Create("iaf_psc_alpha", 10)
pop1.set({"I_e": 376.0})

pop2 = nest.Create("iaf_psc_alpha", 10)

multimeter = nest.Create("multimeter", 10)
multimeter.set({"record_from":["V_m"]})

Populations of neurons are connected via [rules](https://nest-simulator.readthedocs.io/en/v3.3/guides/connection_management.html#connection-management):

- _all_to_all_ (default) - each neuron in pop1 is connected to every neuron in pop2, resulting in 10 \*\* 2 connections.


In [70]:
nest.Connect(pop1, pop2, syn_spec={"weight":20.0})

* _one_to_one_ - the first neuron in pop1 connects to the first neuron in pop2 and so forth.

In [71]:
nest.Connect(pop1, pop2, "one_to_one", syn_spec={"weight":20.0, "delay":1.0})

The multimeter is connected using the default rule

In [72]:
nest.Connect(multimeter, pop2)

### 4. Connecting populatiosn with random connections

In between one_to_one and all_to_all, we can use random connections via the following rule:
* _fixed_indegree_ - creates n random connections for each neuron in the target population post
* _fixed_total_number_ - randomly construct n connections

We can allowing / forbidding self-connections and multiple connections between two neurons by flagging _allow_autapses_ and _allow_multapses_ respectively.

In [73]:
d = 1.0
Je = 2.0
Ke = 20

Ji = -4.0
Ki = 12

conn_dict_ex = {"rule": "fixed_indegree", "indegree": Ke}
conn_dict_in = {"rule": "fixed_indegree", "indegree": Ki}
syn_dict_ex = {"delay": d, "weight": Je}
syn_dict_in = {"delay": d, "weight": Ji}

nest.Connect(epop1, ipop1, conn_dict_ex, syn_dict_ex)
nest.Connect(ipop1, epop1, conn_dict_in, syn_dict_in)

Now each neuron in the target population **ipop1** has 
* Ke incoming random connections 
* chosen from the source population epop1 
* with weight Je and delay d, 

and each neuron in the target population **epop1** has 
* Ki incoming random connections 
* chosen from the source population ipop1 
* with weight Ji and delay d.

### 5. Specifying the behaviour of devices

Devices can be time-configured

In [74]:
pg = nest.Create("poisson_generator")
pg.set({"start": 100.0, "stop": 150.0})

And their output specified using record_to
* ascii (file)
* memory
* screen prints

In [76]:
recdict = {"record_to" : "ascii", "label" : "epop_mp"}
mm1 = nest.Create("multimeter", params=recdict)